# City Schools Report

In [80]:
# Dependencies
import pandas as pd

In [81]:
# Import data
school_data_file_path = "Resources/schools_complete.csv"
student_data_file_path = "Resources/students_complete.csv"

school_data = pd.read_csv(school_data_file_path)
student_data = pd.read_csv(student_data_file_path)


In [82]:
# Join data
school_data_complete = pd.merge(student_data, school_data, how="left", on="school_name")


# Local Government Area Summary

In [83]:
# Outputs not requiring calculation
# Total number of schools
tot_schools = school_data["school_name"].count()

# Total number of students
tot_students = student_data["student_name"].count()

# Total budget
tot_budget = school_data["budget"].sum()

# Average maths score
avg_maths = student_data["maths_score"].mean()

# Average reading score
avg_reading = student_data["reading_score"].mean()


In [84]:
# Calculated outputs
# Percentage of students with a passing maths score
maths_pass = student_data.loc[student_data["maths_score"] >= 50, :]
maths_pass_perc = maths_pass["student_name"].count()/tot_students * 100

# Percentage of students with a passing reading score
reading_pass = student_data.loc[student_data["reading_score"] >= 50, :]
reading_pass_perc = reading_pass["student_name"].count()/tot_students * 100

# Percentage of students with a passing maths score AND a passing reading score
overall_pass = student_data.loc[(student_data["maths_score"] >= 50) &
                                (student_data["reading_score"] >= 50), :]
overall_pass_perc = overall_pass["student_name"].count()/tot_students * 100


In [85]:
# Create dataframe
lga_df = pd.DataFrame({"Total Number of Schools": tot_schools,
                        "Total Number of Students": tot_students,
                        "Total Budget": tot_budget,
                        "Average Maths Score": avg_maths,
                        "Average Reading Score": avg_reading,
                        "% Passing Maths": maths_pass_perc,
                        "% Passing Reading": reading_pass_perc,
                        "% Passing Overall": overall_pass_perc,
                        }, index=[""]
                        )


In [86]:
# Format mapping
lga_df["Total Number of Students"] = lga_df["Total Number of Students"].map("{:,}".format)
lga_df["Total Budget"] = lga_df["Total Budget"].map("${:,.2f}".format)
lga_df["Average Maths Score"] = lga_df["Average Maths Score"].map("{:,.1f}".format)
lga_df["Average Reading Score"] = lga_df["Average Reading Score"].map("{:,.1f}".format)
lga_df["% Passing Maths"] = lga_df["% Passing Maths"].map("{:,.2f}%".format)
lga_df["% Passing Reading"] = lga_df["% Passing Reading"].map("{:,.2f}%".format)
lga_df["% Passing Overall"] = lga_df["% Passing Overall"].map("{:,.2f}%".format)


In [87]:
# Return LGA summary dataframe
lga_df

,Total Number of Schools,Total Number of Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Passing Overall
,15,"39,170","$24,649,428.00",70.3,70.0,86.08%,84.43%,72.81%


# School Summary

In [88]:
# Edit existing school_data data frame to create new school summary dataframe
# Set index as school name
school_summary_df = school_data.set_index("school_name", drop=False)

# Rename columns and remove unneeded columns
school_summary_df = school_summary_df.rename(columns={"type": "School Type",
                                                        "size": "Total Students",
                                                        "budget": "Total Budget",
                                                        "school_name": "School Name"})


In [89]:
# Calculate per student budget
school_summary_df["Budget Per Student"] = school_summary_df["Total Budget"]/school_summary_df["Total Students"]


In [90]:
# Create school groupby object
school_group = school_data_complete.groupby("school_name")

# Create data frame of Average scores by school
school_avg_df = school_group.mean()[["maths_score", "reading_score"]]
school_summary_df = pd.merge(school_summary_df, school_avg_df, how="left", on="school_name")

# Rename columns
school_summary_df = school_summary_df.rename(columns={"maths_score": "Average Maths Score",
                                                        "reading_score": "Average Reading Score"})


In [91]:
# Filter complete dataframe by students who pass only (in 3 categories)
complete_maths_pass_only = school_data_complete.loc[(school_data_complete["maths_score"] >= 50), :]
complete_reading_pass_only = school_data_complete.loc[(school_data_complete["reading_score"] >= 50), :]
complete_overall_pass_only = school_data_complete.loc[(school_data_complete["maths_score"] >= 50) & 
                                                        (school_data_complete["reading_score"] >= 50), :]

# Create series for each category
maths_pass_by_school = complete_maths_pass_only.groupby("school_name")["student_name"].count()
reading_pass_by_school = complete_reading_pass_only.groupby("school_name")["student_name"].count()
overall_pass_by_school = complete_overall_pass_only.groupby("school_name")["student_name"].count()

# Add series to the school summary dataframe
school_summary_df["Total Students Passing Maths"] = maths_pass_by_school
school_summary_df["Total Students Passing Reading"] = reading_pass_by_school
school_summary_df["Total Students Passing Overall"] = overall_pass_by_school

# Calculate percentage of students who passed for each category
school_summary_df["% Passing Maths"] = school_summary_df["Total Students Passing Maths"]/school_summary_df["Total Students"] * 100
school_summary_df["% Passing Reading"] = school_summary_df["Total Students Passing Reading"]/school_summary_df["Total Students"] * 100
school_summary_df["% Passing Overall"] = school_summary_df["Total Students Passing Overall"]/school_summary_df["Total Students"] * 100


In [92]:
# Remove unneeded columns
del school_summary_df["Total Students Passing Maths"]
del school_summary_df["Total Students Passing Reading"]
del school_summary_df["Total Students Passing Overall"]
del school_summary_df["School Name"]
del school_summary_df["School ID"]

# Remove index column title
school_summary_df.index.name = ""

# Sort schools alphabetically
school_format_df = school_summary_df.sort_index()


In [93]:
# Format mapping
school_format_df["Total Students"] = school_format_df["Total Students"].map("{:,}".format)
school_format_df["Total Budget"] = school_format_df["Total Budget"].map("${:,.2f}".format)
school_format_df["Budget Per Student"] = school_format_df["Budget Per Student"].map("${:,.2f}".format)
school_format_df["Average Maths Score"] = school_format_df["Average Maths Score"].map("{:,.1f}".format)
school_format_df["Average Reading Score"] = school_format_df["Average Reading Score"].map("{:,.1f}".format)
school_format_df["% Passing Maths"] = school_format_df["% Passing Maths"].map("{:,.2f}%".format)
school_format_df["% Passing Reading"] = school_format_df["% Passing Reading"].map("{:,.2f}%".format)
school_format_df["% Passing Overall"] = school_format_df["% Passing Overall"].map("{:,.2f}%".format)


In [94]:
# Return school summary dataframe
school_format_df

,School Type,Total Students,Total Budget,Budget Per Student,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Passing Overall
,,,,,,,,,
Bailey High School,Government,"4,976","$3,124,928.00",$628.00,72.4,71.0,91.64%,87.38%,80.08%
Cabrera High School,Independent,"1,858","$1,081,356.00",$582.00,71.7,71.4,90.85%,89.07%,80.79%
Figueroa High School,Government,"2,949","$1,884,411.00",$639.00,68.7,69.1,81.65%,82.81%,67.65%
Ford High School,Government,"2,739","$1,763,916.00",$644.00,69.1,69.6,82.44%,82.22%,67.47%
Griffin High School,Independent,"1,468","$917,500.00",$625.00,71.8,71.2,91.21%,88.49%,81.34%
Hernandez High School,Government,"4,635","$3,022,020.00",$652.00,68.9,69.2,80.95%,81.88%,66.36%
Holden High School,Independent,427,"$248,087.00",$581.00,72.6,71.7,89.93%,88.52%,78.92%
Huang High School,Government,"2,917","$1,910,635.00",$655.00,68.9,68.9,81.69%,81.45%,66.71%
Johnson High School,Government,"4,761","$3,094,650.00",$650.00,68.8,69.0,82.06%,81.98%,67.19%


# Top Performing Schools (By % Overall Passing)

In [95]:
# Sort by passing overall %
top_by_overall_df = school_format_df.sort_values("% Passing Overall", ascending=False)

# return top 5 only
top_by_overall_df = top_by_overall_df.iloc[0:5, :]


In [96]:
# Return top 5 performing schools
top_by_overall_df

,School Type,Total Students,Total Budget,Budget Per Student,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Passing Overall
,,,,,,,,,
Griffin High School,Independent,"1,468","$917,500.00",$625.00,71.8,71.2,91.21%,88.49%,81.34%
Cabrera High School,Independent,"1,858","$1,081,356.00",$582.00,71.7,71.4,90.85%,89.07%,80.79%
Bailey High School,Government,"4,976","$3,124,928.00",$628.00,72.4,71.0,91.64%,87.38%,80.08%
Wright High School,Independent,"1,800","$1,049,400.00",$583.00,72.0,71.0,91.78%,86.67%,79.72%
Rodriguez High School,Government,"3,999","$2,547,363.00",$637.00,72.0,70.9,90.80%,87.40%,79.42%


# Bottom Performing Schools (By % Overall Passing)

In [97]:
# Sort by passing overall %
bottom_by_overall_df = school_format_df.sort_values("% Passing Overall")

# return top 5 only
bottom_by_overall_df = bottom_by_overall_df.iloc[0:5, :]


In [98]:
# Return bottom 5 performing schools
bottom_by_overall_df

,School Type,Total Students,Total Budget,Budget Per Student,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Passing Overall
,,,,,,,,,
Hernandez High School,Government,"4,635","$3,022,020.00",$652.00,68.9,69.2,80.95%,81.88%,66.36%
Huang High School,Government,"2,917","$1,910,635.00",$655.00,68.9,68.9,81.69%,81.45%,66.71%
Johnson High School,Government,"4,761","$3,094,650.00",$650.00,68.8,69.0,82.06%,81.98%,67.19%
Wilson High School,Independent,"2,283","$1,319,574.00",$578.00,69.2,68.9,82.79%,81.30%,67.46%
Ford High School,Government,"2,739","$1,763,916.00",$644.00,69.1,69.6,82.44%,82.22%,67.47%


Maths & Reading Scores By Year

In [99]:
# Create a dataframe with required values
scores_df = school_data_complete[["school_name","year","maths_score","reading_score"]]

# Create an object of group by school and year
scores_group=scores_df.groupby(["school_name", "year"])
scores_by_year = round(scores_group.mean(), 1)

scores_by_year = scores_by_year.reset_index(level=1)

# Use iloc to return scores by year as series
maths_9 = scores_by_year.iloc[0::4, 1]
maths_10 = scores_by_year.iloc[1::4, 1]
maths_11 = scores_by_year.iloc[2::4, 1]
maths_12 = scores_by_year.iloc[3::4, 1]
reading_9 = scores_by_year.iloc[0::4, 2]
reading_10 = scores_by_year.iloc[1::4, 2]
reading_11 = scores_by_year.iloc[2::4, 2]
reading_12 = scores_by_year.iloc[3::4, 2]


# Maths Score by Year

In [100]:
# Create dataframe for the maths scores
maths_scores_df = pd.DataFrame({"Year 9": maths_9,
                                    "Year 10": maths_10,
                                    "Year 11": maths_11,
                                    "Year 12": maths_12})

maths_scores_df.index.name = ""

# Return math scores by year
maths_scores_df




,Year 9,Year 10,Year 11,Year 12
,,,,
Bailey High School,72.5,71.9,72.4,72.7
Cabrera High School,72.3,72.4,71.0,70.6
Figueroa High School,68.5,68.3,68.8,69.3
Ford High School,69.0,69.4,69.2,68.6
Griffin High School,72.8,71.1,71.7,71.5
Hernandez High School,68.6,68.9,69.2,69.0
Holden High School,70.5,75.1,71.6,73.4
Huang High School,69.1,68.5,69.4,68.6
Johnson High School,69.5,68.0,68.6,69.3


# Reading Score by Year

In [101]:
# Create dataframe for the reading scores
reading_scores_df = pd.DataFrame({"Year 9": reading_9,
                                    "Year 10": reading_10,
                                    "Year 11": reading_11,
                                    "Year 12": reading_12})

reading_scores_df.index.name = ""

# Return reading scores by year
reading_scores_df


,Year 9,Year 10,Year 11,Year 12
,,,,
Bailey High School,70.9,70.8,70.3,72.2
Cabrera High School,71.2,71.3,71.2,71.9
Figueroa High School,70.3,67.7,69.2,69.1
Ford High School,69.6,69.0,70.7,68.8
Griffin High School,72.0,70.7,72.4,69.4
Hernandez High School,68.5,70.6,68.4,69.2
Holden High School,71.6,71.1,73.3,70.5
Huang High School,68.7,69.5,68.7,68.7
Johnson High School,68.7,69.3,70.0,68.0


# Scores by School Spending

In [102]:
# Create column with bins
spend_bin = [0, 585, 630, 645, 680]
spend_group_names = ["<$585", "$585-630", "$630-645", "$645-680"]

school_summary_df["Spending Ranges (Per Student)"] = pd.cut(school_summary_df["Budget Per Student"],
                                                spend_bin, labels=spend_group_names, include_lowest=True)

In [103]:
# Create group bin group object and return dataframe
spend_group = school_summary_df.groupby("Spending Ranges (Per Student)")
spend_df = round(spend_group.mean(), 2)
spend_df = spend_df[["Average Maths Score", 
                    "Average Reading Score", 
                    "% Passing Maths", 
                    "% Passing Reading", 
                    "% Passing Overall"]]
spend_df

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Passing Overall
Spending Ranges (Per Student),,,,,
<$585,71.36,70.72,88.84,86.39,76.72
$585-630,72.07,71.03,91.52,87.29,79.88
$630-645,69.85,69.84,84.69,83.76,71.00
$645-680,68.88,69.05,81.57,81.77,66.76


# Scores by School Size

In [104]:
# Create column with bins
size_bin = [0, 1000, 2000, 5000]
size_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_summary_df["School Size"] = pd.cut(school_summary_df["Total Students"],
                                                size_bin, labels=size_names, include_lowest=True)

In [105]:
# Create group bin group object and return dataframe
size_group = school_summary_df.groupby("School Size")
size_df = round(size_group.mean(), 2)
size_df = size_df[["Average Maths Score", 
                    "Average Reading Score", 
                    "% Passing Maths", 
                    "% Passing Reading", 
                    "% Passing Overall"]]
size_df

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Passing Overall
School Size,,,,,
Small (<1000),72.34,71.64,90.81,87.56,79.07
Medium (1000-2000),71.42,70.72,89.85,86.71,78.04
Large (2000-5000),69.75,69.58,84.25,83.30,70.29


# Scores by School Type

In [106]:
# Create group object and return dataframe
type_group = school_summary_df.groupby("School Type")
type_df = round(type_group.mean(), 2)
type_df = type_df[["Average Maths Score", 
                    "Average Reading Score", 
                    "% Passing Maths", 
                    "% Passing Reading", 
                    "% Passing Overall"]]
type_df

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Passing Overall
School Type,,,,,
Government,69.83,69.68,84.46,83.59,70.70
Independent,71.37,70.72,89.20,86.25,76.97


# School Summary Correlations

In [108]:
school_summary_df.corr()

,Total Students,Total Budget,Budget Per Student,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Passing Overall
Total Students,1.000000,0.998340,0.636490,-0.387786,-0.464706,-0.382718,-0.383160,-0.379057
Total Budget,0.998340,1.000000,0.674560,-0.407924,-0.476158,-0.406312,-0.400378,-0.400765
Budget Per Student,0.636490,0.674560,1.000000,-0.576831,-0.522422,-0.572934,-0.536663,-0.555479
Average Maths Score,-0.387786,-0.407924,-0.576831,1.000000,0.940124,0.981703,0.945183,0.975811
Average Reading Score,-0.464706,-0.476158,-0.522422,0.940124,1.000000,0.922345,0.947796,0.942211
% Passing Maths,-0.382718,-0.406312,-0.572934,0.981703,0.922345,1.000000,0.940587,0.989927
% Passing Reading,-0.383160,-0.400378,-0.536663,0.945183,0.947796,0.940587,1.000000,0.976084
% Passing Overall,-0.379057,-0.400765,-0.555479,0.975811,0.942211,0.989927,0.976084,1.000000
